In [ ]:
# ansel lim
# 3 november 2021
# use the data from https://data.gov.sg/dataset/listing-of-licensed-supermarkets and API at https://www.onemap.gov.sg/docs/#search to get latlong of supermarkets

In [ ]:
import pandas as pd
import re
import requests
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/6242_PROJECT/data

/content/gdrive/MyDrive/6242_PROJECT/data


In [ ]:
%ls

 amenities.csv                    eating-establishments.kml
 bus_stops.csv                    general-information-of-schools.csv
 carparks.csv                     listing-of-supermarkets.csv
 data-download.png                parks.csv
 data_hawker.csv                  parks-kml.kml
 data_hawker.gsheet               processed/
 data_malls.csv                   secondary_schools.csv
 data_MRT.csv                     secondary_schools.gsheet
 data_prischools.csv              supermarkets.csv
 data_prischools.gsheet           taxi_stands.csv
'eating_establishments (1).csv'   ura.csv
 eating_establishments.csv        ura_join_bus_stops.csv


In [ ]:
supermarkets = pd.read_csv("./listing-of-supermarkets.csv")

In [ ]:
supermarkets

,licence_numbers,business_name,name_of_license,premise_address
0,B02008E000,SHENG SIONG SUPERMARKET,SHENG SIONG SUPERMARKET PTE LTD,"845 YISHUN STREET 81 #01-184, S(760845)"
1,B02011P000,GIANT,COLD STORAGE SINGAPORE (1983) PTE LTD,"524A JELAPANG ROAD #03-13/18, GREENRIDGE SHOPP..."
2,B02012N000,COLD STORAGE,COLD STORAGE SINGAPORE (1983) PTE LTD,"768 WOODLANDS AVENUE 6 #01-34, WOODLANDS MART ..."
3,B02015J000,SHENG SIONG SUPERMARKET,SHENG SIONG SUPERMARKET PTE LTD,"122 ANG MO KIO AVENUE 3 #01-1753,#01-1757,#01-..."
4,B02017C000,SHENG SIONG SUPERMARKET,SHENG SIONG SUPERMARKET PTE LTD,"301 WOODLANDS STREET 31 #01-217, S(730301)"
...,...,...,...,...
602,W70029L000,FAIRPRICE,NTUC FAIRPRICE CO-OPERATIVE LTD,"170 STIRLING ROAD #01-1147, S(140170)"
603,W72014K000,GIANT,COLD STORAGE SINGAPORE (1983) PTE LTD,"86 REDHILL CLOSE #01-610, S(150086)"
604,W74072X000,GIANT,COLD STORAGE SINGAPORE (1983) PTE LTD,"8 TELOK BLANGAH CRESCENT #01-171, S(090008)"
605,W89166E000,COLD STORAGE,COLD STORAGE SINGAPORE (1983) PTE LTD,"176 ORCHARD ROAD #B1-09/11, CENTREPOINT SHOPPI..."


In [ ]:
def getLatLong(x):
  address = x['premise_address']
  postal_code = re.findall(r'S[(][0-9]{6}[)]',address)[0].lstrip("S(").rstrip(")")
  matches = requests.get("https://developers.onemap.sg/commonapi/search?searchVal={}&returnGeom=Y&getAddrDetails=Y&pageNum=1".format(postal_code)).json()['results']
  try:
    first_match = matches[0]
    lat,long = first_match['LATITUDE'],first_match['LONGITUDE']
    isLatLongAvailable='True'
    return (lat,long,isLatLongAvailable)
  except IndexError:
    isLatLongAvailable='False'
    return ("unknown","unknown",isLatLongAvailable)

In [ ]:
getLatLong(supermarkets.iloc[0,:])

('1.41599814748468', '103.835053564214', 'True')

In [ ]:
#supermarkets.loc[:,['lat','long',"isLatLongAvailable"]] = supermarkets.apply(getLatLong,axis=1,result_type='expand')

supermarkets=pd.concat([supermarkets,supermarkets.apply(getLatLong,axis=1,result_type='expand').rename(mapper={0:'lat',1:'long',2:'isLatLongAvailable'},axis=1)],axis=1)

In [ ]:
supermarkets

,licence_numbers,business_name,name_of_license,premise_address,lat,long,isLatLongAvailable
0,B02008E000,SHENG SIONG SUPERMARKET,SHENG SIONG SUPERMARKET PTE LTD,"845 YISHUN STREET 81 #01-184, S(760845)",1.41599814748468,103.835053564214,True
1,B02011P000,GIANT,COLD STORAGE SINGAPORE (1983) PTE LTD,"524A JELAPANG ROAD #03-13/18, GREENRIDGE SHOPP...",1.38540569310644,103.766228468185,True
2,B02012N000,COLD STORAGE,COLD STORAGE SINGAPORE (1983) PTE LTD,"768 WOODLANDS AVENUE 6 #01-34, WOODLANDS MART ...",1.44564787003157,103.798162285805,True
3,B02015J000,SHENG SIONG SUPERMARKET,SHENG SIONG SUPERMARKET PTE LTD,"122 ANG MO KIO AVENUE 3 #01-1753,#01-1757,#01-...",1.37018860668947,103.843413231524,True
4,B02017C000,SHENG SIONG SUPERMARKET,SHENG SIONG SUPERMARKET PTE LTD,"301 WOODLANDS STREET 31 #01-217, S(730301)",1.4315052703273,103.773515659097,True
...,...,...,...,...,...,...,...
602,W70029L000,FAIRPRICE,NTUC FAIRPRICE CO-OPERATIVE LTD,"170 STIRLING ROAD #01-1147, S(140170)",1.29091128182876,103.803120797076,True
603,W72014K000,GIANT,COLD STORAGE SINGAPORE (1983) PTE LTD,"86 REDHILL CLOSE #01-610, S(150086)",1.2869200622963,103.818673592235,True
604,W74072X000,GIANT,COLD STORAGE SINGAPORE (1983) PTE LTD,"8 TELOK BLANGAH CRESCENT #01-171, S(090008)",1.27779539068994,103.819341303524,True
605,W89166E000,COLD STORAGE,COLD STORAGE SINGAPORE (1983) PTE LTD,"176 ORCHARD ROAD #B1-09/11, CENTREPOINT SHOPPI...",1.30145045537088,103.840034074858,True


In [ ]:
supermarkets = supermarkets.reset_index()

In [ ]:
supermarkets

,index,licence_numbers,business_name,name_of_license,premise_address,lat,long,isLatLongAvailable
0,0,B02008E000,SHENG SIONG SUPERMARKET,SHENG SIONG SUPERMARKET PTE LTD,"845 YISHUN STREET 81 #01-184, S(760845)",1.41599814748468,103.835053564214,True
1,1,B02011P000,GIANT,COLD STORAGE SINGAPORE (1983) PTE LTD,"524A JELAPANG ROAD #03-13/18, GREENRIDGE SHOPP...",1.38540569310644,103.766228468185,True
2,2,B02012N000,COLD STORAGE,COLD STORAGE SINGAPORE (1983) PTE LTD,"768 WOODLANDS AVENUE 6 #01-34, WOODLANDS MART ...",1.44564787003157,103.798162285805,True
3,3,B02015J000,SHENG SIONG SUPERMARKET,SHENG SIONG SUPERMARKET PTE LTD,"122 ANG MO KIO AVENUE 3 #01-1753,#01-1757,#01-...",1.37018860668947,103.843413231524,True
4,4,B02017C000,SHENG SIONG SUPERMARKET,SHENG SIONG SUPERMARKET PTE LTD,"301 WOODLANDS STREET 31 #01-217, S(730301)",1.4315052703273,103.773515659097,True
...,...,...,...,...,...,...,...,...
602,602,W70029L000,FAIRPRICE,NTUC FAIRPRICE CO-OPERATIVE LTD,"170 STIRLING ROAD #01-1147, S(140170)",1.29091128182876,103.803120797076,True
603,603,W72014K000,GIANT,COLD STORAGE SINGAPORE (1983) PTE LTD,"86 REDHILL CLOSE #01-610, S(150086)",1.2869200622963,103.818673592235,True
604,604,W74072X000,GIANT,COLD STORAGE SINGAPORE (1983) PTE LTD,"8 TELOK BLANGAH CRESCENT #01-171, S(090008)",1.27779539068994,103.819341303524,True
605,605,W89166E000,COLD STORAGE,COLD STORAGE SINGAPORE (1983) PTE LTD,"176 ORCHARD ROAD #B1-09/11, CENTREPOINT SHOPPI...",1.30145045537088,103.840034074858,True


In [ ]:
supermarkets

,index,licence_numbers,business_name,name_of_license,premise_address,lat,long,isLatLongAvailable
0,0,B02008E000,SHENG SIONG SUPERMARKET,SHENG SIONG SUPERMARKET PTE LTD,"845 YISHUN STREET 81 #01-184, S(760845)",1.41599814748468,103.835053564214,True
1,1,B02011P000,GIANT,COLD STORAGE SINGAPORE (1983) PTE LTD,"524A JELAPANG ROAD #03-13/18, GREENRIDGE SHOPP...",1.38540569310644,103.766228468185,True
2,2,B02012N000,COLD STORAGE,COLD STORAGE SINGAPORE (1983) PTE LTD,"768 WOODLANDS AVENUE 6 #01-34, WOODLANDS MART ...",1.44564787003157,103.798162285805,True
3,3,B02015J000,SHENG SIONG SUPERMARKET,SHENG SIONG SUPERMARKET PTE LTD,"122 ANG MO KIO AVENUE 3 #01-1753,#01-1757,#01-...",1.37018860668947,103.843413231524,True
4,4,B02017C000,SHENG SIONG SUPERMARKET,SHENG SIONG SUPERMARKET PTE LTD,"301 WOODLANDS STREET 31 #01-217, S(730301)",1.4315052703273,103.773515659097,True
...,...,...,...,...,...,...,...,...
602,602,W70029L000,FAIRPRICE,NTUC FAIRPRICE CO-OPERATIVE LTD,"170 STIRLING ROAD #01-1147, S(140170)",1.29091128182876,103.803120797076,True
603,603,W72014K000,GIANT,COLD STORAGE SINGAPORE (1983) PTE LTD,"86 REDHILL CLOSE #01-610, S(150086)",1.2869200622963,103.818673592235,True
604,604,W74072X000,GIANT,COLD STORAGE SINGAPORE (1983) PTE LTD,"8 TELOK BLANGAH CRESCENT #01-171, S(090008)",1.27779539068994,103.819341303524,True
605,605,W89166E000,COLD STORAGE,COLD STORAGE SINGAPORE (1983) PTE LTD,"176 ORCHARD ROAD #B1-09/11, CENTREPOINT SHOPPI...",1.30145045537088,103.840034074858,True


In [ ]:
supermarkets.to_csv("./supermarkets.csv")